In [1]:
from flask import Flask
import base64
import pymongo
from pymongo.database import Database 
import matplotlib.pyplot as plt
from io import BytesIO
import os
from PIL import Image
import requests
import json

<font size=5> Setup Database Connection</font>

In [2]:
def mongodb_init():
    # connect to mongodb
    mongo = pymongo.MongoClient(host='18.162.214.19', port=27017, username="root", password="1647#4hkust", authSource='admin')
    print('数据库当前的databases: ', mongo.list_database_names())
    return mongo

def get_db(mongo, db_name):
    db = Database(name=db_name, client=mongo)
    print('获取/创建库：', db.name)
    return db

# API below
app = Flask(__name__)
mongo = mongodb_init()
db = get_db(mongo, 'chen_db')

数据库当前的databases:  ['admin', 'chen_db', 'config', 'local']
获取/创建库： chen_db


<font size=5> Helper Functions </font>

In [9]:
def compress_image(infile, outfile='', mb=30, step=15, quality=60, printable=True):
    """
    不改变图片尺寸压缩到指定大小
    :param infile: 压缩源文件
    :param outfile: 压缩文件保存地址
    :param mb: 压缩比率
    :param step: 每次调整的压缩比率
    :param quality: 初始压缩比率
    :param print: 是否打印
    :return: 压缩文件地址，压缩文件大小
    """
    o_size = os.path.getsize(infile) / 1024
    if printable: print(f"original size: {o_size:.2f} KB")
    if o_size <= 5:
        return infile
    outfile = get_outfile(infile, outfile)
    
    while o_size > o_size * mb/100:
        im = Image.open(infile)
        im.save(outfile, quality=quality)
        if quality - step < 0:
            break
        quality -= step
        o_size = os.path.getsize(outfile) / 1024
        
    return outfile, o_size


def get_outfile(infile, outfile):
    if outfile:
        return outfile
    dir, suffix = os.path.splitext(infile)
    outfile = '{}-out{}'.format(dir, suffix)
    return outfile


def Image_Encode_Compress(image_URL, printable=True):
    # 压缩图片
    compressed_image, compressed_size = compress_image(infile=image_URL, printable=printable)
    # 读取本地图片文件
    if printable: print(f"Size after compress: {compressed_size:.2f} KB")
    
    with open(compressed_image, 'rb') as input:
        image_data = input.read()  # 图片的二进制数据

    # 将二进制图片编码为 base64 格式
    base64_data_bytes = base64.b64encode(image_data)  # 图片的 bytes 类型
    base64_data_str = base64_data_bytes.decode()  # 图片的字符串类型
    
    # 删除压缩的图片
    os.remove(compressed_image)
    return base64_data_str


def Image_Encode(image_URL):
    with open(image_URL, 'rb') as input:
        image_data = input.read()  # 图片的二进制数据

    # 将二进制图片编码为 base64 格式
    base64_data_bytes = base64.b64encode(image_data)  # 图片的 bytes 类型
    base64_data_str = base64_data_bytes.decode()  # 图片的字符串类型
    return base64_data_str


def Image_show(image_URL):
    with open(image_URL, 'rb') as input:
        image_data = input.read()  # 图片的二进制数据
    # 展示图片
    img_data = plt.imread(BytesIO(image_data), format="JPG")
    import cv2
    img_cv2 = cv2.cvtColor(img_data, cv2.COLOR_RGB2BGR)
    cv2.imshow('Image', img_cv2)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    

def upload(Title, Text, Price, Fields, Images, Latitude, Longitude):
    url = 'http://localhost:8080/new-post'
    # url = "http://18.162.214.19/chen/new-post"

    body = {    'PostOwner': '1030920919@qq.com',
                'Title': Title,
                'Text': Text,
                'Auction': False,
                'LostFound': False,
                'Price': Price,
                'Fields': Fields,
                'Images': Images,
                "Latitude" : Latitude,
                "Longitude" : Longitude,
            }
    r = requests.post(url, json=body)
    return r.json()["Success"]


def upload_folder(folder_path, printable=True):
    Images = []
     # 加载所有图片
    for filename in os.listdir(folder_path):
        if filename.lower().endswith( ('.png', '.jpg', '.jpeg') ):
            image_path = os.path.join(folder_path, filename)
            # 是否压缩
            # Images.append( Image_Encode_Compress(image_path) )
            Images.append( Image_Encode_Compress(image_path, printable=printable) )
        
    # 加载所有文件列
    data_path = os.path.join(folder_path, "data.json")
    with open(data_path, newline='') as json_file:
        json_data = json.load(json_file)
    
    Title = json_data["Title"]
    Text = json_data["Text"]
    Price = json_data["Price"]
    Fields = json_data["Fields"]
    Latitude = json_data["Location"]["coordinates"][0]
    Longitude = json_data["Location"]["coordinates"][1]
    
    success = upload(Title, Text, Price, Fields, Images, Latitude, Longitude)
    return success

<font size=5> Download all Posts from Database </font>

In [4]:
import io

# 保存所有的post到本地
Posts = db.Posts.find()
if Posts:
    folder_Count = 0
    for Post in Posts:
        folder_Count += 1
        
        # 创建文件夹
        folder_path = "../image/"+ str(folder_Count)
        if not os.path.exists(folder_path):
            os.makedirs(folder_path)
            
        # 获取图片和文字
        data = {
            "PID": str(Post['_id']), 
            "Title": Post.get("Title").strip("\n"),
            "Text": Post.get("Text").strip("\n"),
            "Price": str(Post.get("Price")),
            "Fields": Post.get("Fields"),
            "Location": Post.get("Location"),
            }
        Images_base64 = Post.get("Images")
        # 存图片
        if Images_base64: # not None
            image_Count = 0
            # 遍历所有图片
            for Image_base64 in Images_base64:
                image_Count += 1
                # 转换图片
                decoded_image_data = base64.b64decode(Image_base64)
                image = Image.open(io.BytesIO(decoded_image_data))

                image_path = os.path.join(folder_path, str(image_Count) + ".png")
                image.save(image_path)
                
        # 存data
        data_path = os.path.join(folder_path,  "data.json")
        with open(data_path, 'w', newline='') as json_file:
            json.dump(data, json_file, indent=4)

<font size=5> Upload all Posts to Database </font>

In [12]:
# Run above folder first to get all Posts from database. or unzip image.zip as the following structure.

# Server/API/upload.ipynb
# Server/image/1/1.png
# Server/image/1/data.json

folder_Count = 1
success_Count = 0 
folder_path = "../image/"+ str(folder_Count)

# 遍历所有文件夹
while os.path.exists(folder_path):
    # 上传文件夹内的对应文件
    print(folder_path)
    success = upload_folder(folder_path, printable=False)
    if success: 
        success_Count += 1
    
    # 更新folder path
    folder_Count += 1
    folder_path = "../image/"+ str(folder_Count)

../image/1
../image/2
../image/3
../image/4
../image/5
../image/6
../image/7
../image/8
../image/9
../image/10
../image/11
../image/12
../image/13
../image/14
../image/15
../image/16
../image/17
../image/18
../image/19
../image/20
../image/21
../image/22
../image/23
../image/24
../image/25
../image/26
../image/27


In [23]:
print(f"success:{success_Count}/{folder_Count-1}")

success:18/18


<font size=5> Upload A Single New Post </font>

In [39]:
# set the image directories
imageUrl_1 = '../image/' + '1.webp'
imageUrl_2 = '../image/' + '2.webp'
imageUrl_3 = '../image/' + '3.webp'
imageUrl_4 = '../image/' + '4.webp'

Images = []
Images.append( Image_Encode_Compress(imageUrl_1) )
Images.append( Image_Encode_Compress(imageUrl_2) )
Images.append( Image_Encode_Compress(imageUrl_3) )
Images.append( Image_Encode_Compress(imageUrl_4) )

# set parameters
Title = "Bauer Girls Helmet."

Text = \
"""
Hello,
I'm selling a good condition helmet, size is 6.
"""

Price = "10"
Fields = []

success = upload(Title, Text, Price, Fields, Images)
print(success)

original size: 28.61 KB
Size after compress: 10.47 KB
original size: 25.89 KB
Size after compress: 9.94 KB
original size: 27.57 KB
Size after compress: 10.45 KB


<font size=5> Upload A Single Folder </font>

In [17]:
folder_path = '../image/' + '11'
success = upload_folder(folder_path, printable=True)
print(success)

original size: 354.92 KB
Size after compress: 354.92 KB
original size: 231.59 KB
Size after compress: 231.59 KB
True


In [14]:
data_path = os.path.join("../image/3", "data.json")
with open(data_path, newline='') as json_file:
    json_data = json.load(json_file)
print(json_data)

Title = json_data["Title"]
Text = json_data["Text"]
Price = json_data["Price"]
Fields = json_data["Fields"]

print(Title, Text, Price, Fields)

{'PID': '65cbbbe0e2ce5bb93a77bfad', 'Title': 'Custom Fiend/Cult BMX Build (MUST SEE).', 'Text': 'For sale is a fully custom BMX bike with all top-of-the-line parts in incredible condition!\nThe parts list is as follows:\nFrame: Fiend Reynolds V2 (21” TT)\nHandlebars: Cult Crew (9” Rise)\nForks: Fiend Invest (20mm Offset)\nStem: Fiend Morrow V3 (48mm Reach)\nHeadset: WTP Compact\nGrips: Cult x Vans Waffle\nSeat: Cult x Vans Pivotal\nSeatpost: Odyssey Pivotal\nSprocket: Éclat RS (25T)\nCranks: WeThePeople Logic (165mm)\nBB: Éclat Mid Bottom Bracket (22mm)\nPedals: Federal Command Pedals\nChain: Éclat Stroke (Half-Link)\nFront Wheel: Cult Crew V2\nRear Wheel: Cult Crew Cassette V2 (RHD)\nTires: Éclat Morrow (2.4” Wide)\n———SELLING FOR $750.00———\n***Feel free to checkout my other ads for more top brand BMX parts (new condition & great deals)***\nIf you have any further questions you can message me at +1 (204) 620-5650 or via Kijiji Messenger.', 'Price': '750', 'Fields': ['Cars & Vehicles'